In [4]:

# coding: utf-8

# In[65]:


from lxml import etree
from collections import OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pickle
import numpy as np



# In[41]:


datamap=OrderedDict()
datamap["android"]="android.stackexchange.com"
datamap["dba"]="dba.stackexchange.com"
datamap["softwareEng"]="softwareengineering.stackexchange.com"
datamap["serverfault"]= "serverfault.com"
datamap["superuser"]="superuser.com"
datamap["stackoverflow"]="stackoverflow.com-"


# In[59]:


def read_tags(filename):
    logtags=[]
    for row in open(filename,"r"):
        logtags.append(row.split(",")[0].strip().lower())   
    return logtags

def read_prog_lang(filename):
    data=OrderedDict()
    f=open(filename,"r")
    for l in f:
        row=l.split("\t")
        key=row[0].lower().strip()
        data[key]=[]
        for element in row[1].split(","):
            data[key].append(element.strip().lower())
    return data

def atleast_one(a, b):
    return not set(a).isdisjoint(b)

def refine_tags(tags):
    if tags!=None:
        l=tags.split('><')
        l=[i.replace('>','').replace('<','').lower() for i in l]
        return l
    else:
        return ["???"]

def init_data(taglist):
    data=OrderedDict()
    for tag in taglist:
        data[tag]=OrderedDict()
        for fname in datamap:
            data[tag][fname]=0
    return data


# In[49]:


logtags=list(set(read_tags("logging - Sheet3.csv")))

prog_tags=read_prog_lang("logging - Sheet4.tsv")

for i in prog_tags:
    logtags.extend(prog_tags[i])
logtags=list(set(logtags))

# del logtags[logtags.index("")]


# In[57]:


def common_tag_csv(data):
    f=open("Sheet-2.csv","w")
    f.write(",")
    for fname in datamap:
        f.write(fname+",")
    f.write("\n")
    for tag in data:
        f.write(tag+",")
        for element in data[tag]:
            f.write(str(data[tag][element])+",")
        f.write("\n")
    f.close()
    
    fr=open("Sheet-2.csv","r")
    fw=open("Modified-Sheet-2.csv","w")
    fw.write(fr.readline())
    for l in fr:
        row=l.split(",")
        temp=Counter(row)
        print(temp)
        if temp['0'] <=4:
            fw.write(l)
    fw.close()
    fr.close()
        
        


# In[51]:


def count_tags(taglist):
    data=init_data(taglist)
    #print(data)
    for fname in datamap:
        print(fname)
        if fname=="stackoverflow" and datamap[fname]=="stackoverflow.com-":
            datamap[fname]=datamap[fname]+"Tags"
        f=open("data/"+datamap[fname]+"/Tags.xml","rb")
        next(f)
        next(f)
        count=0
        for l in f:
            try:
                rows=etree.XML(l)  
                if  rows.get("TagName").lower() in taglist:
                    tagname=rows.get("TagName").lower()
                    question=rows.get("Count")
                    data[tagname][fname]=question
                    count=count+int(question)
                    #print(tagname,question)
            except:
                print(l)
               

        print(count)
    return data


# In[58]:


# data=count_tags(logtags)
# common_tag_csv(data)


# In[66]:


def plot_monthly_bargraph(plt,data,title,label,opr):
    # plt.close()
    # plt.clf()
    if opr==1:
        color='b'
    else:
        color='g'
    y=[data[i] for i in data]
    x=[i for i in range(len(data))]
    xticks=[i.split('-')[0] for i in data]
    plt.bar(x,y,label=label,color=color)
    gap=1
    plt.xticks(x[::gap],xticks[::gap],rotation=30)
    #plt.title(title.upper())
    plt.ylabel("No. of Questions")
    return plt
    # if opr==1:
    #     plt.savefig("Images/"+title+".png")
    # else:
    #     plt.show()
    # plt.clf()

    
def plot_monthly_cumgraph(plt,data,title,label,opr):
    if opr==1:
        color='b'
        marker=","
    else:
        color='g'
        marker="o"
    y=np.cumsum([data[i] for i in data])
    x=[i for i in range(len(data))]
    xticks=[i.split('-')[0] for i in data]
    plt.plot(x,y,color=color)
    plt.scatter(x,y,marker=marker,label=label,color=color)
    gap=1
    plt.xticks(x[::gap],xticks[::gap],rotation=30)
    #plt.title(title.upper())
    plt.ylabel("No. of Questions")
    # if opr==1:
    #     plt.savefig("Images/"+title+".png")
    # else:
    #     plt.show()
    # plt.clf()
    return plt

def init_o_data():
    data=OrderedDict()
    start_year=2008
    start_month=8
    end_year=2017
    end_month=12
    month=1
    for year in range(start_year,end_year+1):
       # for month in range(start_month,end_month+1):
            #print(year,month)
            temp=str(month)
            if len(temp)==1:
                temp="0"+temp
            data[str(year)]=[]
        #start_month=1
    return data


# In[61]:


def distribute_posts(website,logtags):
    if fname=="stackoverflow" and datamap[fname]=="stackoverflow.com-":
        datamap[fname]=datamap[fname]+"Posts"   
    
    f=open("data/"+datamap[website]+"/Posts.xml","rb")
    next(f)
    next(f)
    data=init_o_data()
    data_all=init_o_data()
    count=0
    for l in f:
        try:
            row=etree.XML(l)      
            posttypeid=row.get("PostTypeId") 
            acceptedanswerID=row.get("AcceptedAnswerId")
            if posttypeid=="1":
                tags=refine_tags(row.get("Tags"))
                if atleast_one(tags,logtags):
                        date=row.get("CreationDate")
                        key=date.split("T")[0][0:4]
                        if key not in data_all:
                            data_all[key]=[]
                        data_all[key].append((int(row.get("ViewCount")),int(row.get("Id"))))
            
            if posttypeid=="1" and acceptedanswerID!=None:
                # Means it has an accepted answer
                
                if atleast_one(tags,logtags):
                    # Atleast one logging tag

                    #print(tags)
                    date=row.get("CreationDate")
                    key=date.split("T")[0][0:4]
                    if key not in data:
                        data[key]=[]
                    data[key].append((int(row.get("ViewCount")),int(row.get("Id"))))
        except:
            print(l)
    return data, data_all


# In[68]:


for fname in list(datamap.keys()):
   
    data, data_all=distribute_posts(fname,logtags)
    pickle.dump(data,open(fname+".view.pickle","wb"))
    pickle.dump(data_all,open(fname+".view.all.pickle","wb"))



b'</posts>'
b'</posts>'
b'</posts>'
b'</posts>'
b'</posts>'
b'</posts>'


In [16]:
for fname in list(datamap.keys()):
    f=open(fname+"_view.csv","w")
    f.write("Sno,ID,Total\n")
    data=pickle.load(open(fname+".view.pickle","rb"))
    data_all=pickle.load(open(fname+".view.all.pickle","rb"))
    data=[]
    for i in data_all:
        data.extend(data_all[i])
    print(data)
    info=sorted(data, key=lambda x: x[0])[-6:-1]
    print(info)
    c=1
    for l in info:
        f.write(str(c)+","+str(l[1])+","+str(l[0])+"\n")
        c+=1
    f.close()

[(18989, 157), (103103, 941), (893, 6623), (541, 6826), (340, 7297), (197, 7704), (13937, 8632), (171, 8764), (17482, 13992), (237, 14260), (334450, 14430), (516, 14564), (297, 14763), (179, 15100), (4830, 16080), (276, 16278), (60385, 16905), (9633, 17282), (4851, 17572), (1114, 18013), (8490, 19809), (305, 20818), (1610, 21516), (298, 22043), (2307, 24099), (19749, 26080), (5179, 26340), (8811, 26449), (15441, 26463), (574, 27555), (30761, 27586), (2552, 28325), (702, 28848), (20292, 28857), (1251, 30144), (9425, 30207), (750, 32165), (1452, 32206), (1077, 32541), (1951, 32817), (26335, 33216), (945, 33321), (1095, 34607), (508, 35258), (7102, 35702), (270, 36894), (26, 36931), (14291, 37170), (1165, 37447), (4213, 37487), (4955, 39527), (51, 39562), (1314, 40080), (1539, 40421), (45666, 41642), (5053, 41825), (1751, 45053), (319, 45112), (132, 45572), (28930, 48279), (1047, 48974), (2263, 49813), (2378, 51274), (10648, 51733), (1719, 51974), (46, 52010), (302, 53404), (4063, 53477),